In [ ]:
from keras.datasets.mnist import load_data
(X_train,y_train), (X_test, y_test)  = load_data()

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
import matplotlib.pyplot as plt
for i in range(8):
    plt.subplot(4, 2, 1+i)
    plt.axis('off')
    plt.imshow(X_train[i], cmap = 'viridis')
plt.show()

In [ ]:
from tensorflow.keras.layers import Dense, LeakyReLU, Reshape, BatchNormalization
import tensorflow as tf
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, Dropout, Flatten


from tensorflow.keras.layers import Conv2D, Dropout, Flatten, Dense
from tensorflow.keras.utils import plot_model
from keras.layers import Conv2DTranspose

## Discriminator

In [ ]:
def Discriminator(shape = (28,28,1)):
    model = Sequential()
    model.add( Conv2D(64, (3,3), strides = (2,2), padding = 'same', input_shape = shape ) )
    model.add( LeakyReLU(alpha = 0.2) )
    model.add( Dropout(0.4) )

    model.add( Conv2D(64, (3,3), strides = (2,2), padding = 'same' ) )
    model.add( LeakyReLU(alpha = 0.2) )
    model.add( Dropout(0.4) )

    model.add( Flatten() )
    model.add( Dense(1, activation = 'sigmoid' ))

    opt = Adam(learning_rate = 0.0002, beta_1 = 0.5)
    model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['accuracy'])

    return model

In [ ]:
model = Discriminator()
model.summary()

## Loading Real Samples

In [ ]:
from keras.datasets.mnist import load_data
import numpy as np

def load_real_samples():
    (X_train,y_train), (X_test, y_test)  = load_data()
    X = np.expand_dims(X_train, axis = -1)
    X = X.astype('float32')
    X = X/255
    return X

In [ ]:
real_samples = load_real_samples()
real_samples.shape
#print(real_samples)

## Generate Real Samples

In [ ]:
def generate_real_samples(dataset, n_samples):
    ix = np.random.randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = np.ones((n_samples, 1))

    return X,y

In [ ]:
real_samples = load_real_samples()
real_samples, y = generate_real_samples(real_samples, 256)
print(real_samples.shape)

fig, axes = plt.subplots(3, 5, figsize=(10, 6))
axes = axes.flatten()
for i in range(15):
    axes[i].axis('off')
    axes[i].imshow(real_samples[i, :, :, 0])


plt.tight_layout()
plt.show()

## Generate Noise Samples

In [ ]:
def generate_noise_samples(n_samples):
    X = np.random.rand(28*28*n_samples)
    X = X.reshape((n_samples,28,28,1))
    y = np.zeros((n_samples, 1))

    return X,y

In [ ]:
noise_samples,y = generate_noise_samples(256)
print(noise_samples.shape)

fig, axes = plt.subplots(3, 5, figsize=(10, 6))
axes = axes.flatten()
for i in range(15):
    axes[i].axis('off')
    axes[i].imshow(noise_samples[i, :, :, 0])


plt.tight_layout()
plt.show()

## Training Discriminator

In [ ]:
def train_discriminator(model, dataset, n_iter = 100, n_batch = 256):
    half_batch = int(n_batch/2)

    for i in range(n_iter):
        X_real, y_real = generate_real_samples(dataset, half_batch)
        real_loss, real_acc = model.train_on_batch(X_real, y_real)

        X_noise, y_noise = generate_noise_samples(half_batch)
        noise_loass, noise_acc = model.train_on_batch(X_noise, y_noise)

        #print('>%d real=%.0f%% noise=%.0f%%' % (i+1, real_acc*100, noise_acc*100))

In [ ]:
model = Discriminator()
dataset = load_real_samples()
train_discriminator(model, dataset)

## Generator

In [ ]:
def Generator(latent_dim):
    model = Sequential()

    model.add( Dense(128*7*7, input_dim = latent_dim ))
    model.add( LeakyReLU(alpha = 0.2) )
    model.add( Reshape((7,7,128)) )

    model.add( Conv2DTranspose(128, (4,4), strides = (2,2), padding = 'same' ))
    model.add( LeakyReLU(alpha = 0.2) )

    model.add( Conv2DTranspose(128, (4,4), strides = (2,2), padding = 'same' ))
    model.add( LeakyReLU(alpha = 0.2) )

    model.add( Conv2D(1, (7,7), activation = 'sigmoid', padding = 'same' ))
    return model

In [ ]:
model = Generator(100)
model.summary()

## Generating Latent Points

In [ ]:
def generate_latent_points(latent_dim, n_samples):
    X_input = np.random.randn( latent_dim*n_samples )
    X_input = X_input.reshape( n_samples, latent_dim )

    return X_input

## Generating Synthetic Samples

In [ ]:
def generate_synthetic_samples(model, latent_dim, n_samples):
    X_input = generate_latent_points(latent_dim, n_samples)

    X = model.predict(X_input)
    y = np.zeros((n_samples, 1))

    return X,y

## Plotting Synthetic Samples

In [ ]:
latent_dim = 100
n_samples = 256

model = Generator(latent_dim)
X, y = generate_synthetic_samples(model, latent_dim, n_samples)


fig, axes = plt.subplots(3, 5, figsize=(10, 6))
axes = axes.flatten()
for i in range(15):
    axes[i].axis('off')
    axes[i].imshow(X[i, :, :, 0])


plt.tight_layout()
plt.show()

## GAN

In [ ]:
def GAN(g_model, d_model):
    d_model.trinable = False

    model = Sequential()
    model.add(g_model)
    model.add(d_model)

    opt = Adam(learning_rate = 0.0002, beta_1 = 0.5)
    model.compile(loss = 'binary_crossentropy', optimizer = opt)
    return model

In [ ]:
d_model = Discriminator()
g_model = Generator(100)

gan = GAN(g_model, d_model)
gan.summary()

## Training GAN

In [ ]:
def tain_GAN(gan_model, latent_dim, n_epochs = 100, n_batch = 256):

    for i in range(n_epochs):
        X_gan = generate_latent_points(latent_dim, n_batch)
        y_gan = np.ones( (n_batch, 1) )

        gan_model.train_on_batch(X_gan, y_gan)
        return gan_model

## Training Generator and Discriminator

In [ ]:
def train_gen_dis(g_model, d_model, gan_model, dataset, latent_dim, n_epoches = 40, n_batch = 256):
    batch_per_epoch = int(dataset.shape[0]/n_batch)
    half_batch = int(n_batch / 2)

    for i in range(n_epoches):
        for j in range(batch_per_epoch):

            X_real, y_real = generate_real_samples(dataset, half_batch)

            X_syn, y_syn = generate_synthetic_samples(g_model, latent_dim, half_batch)




            X, y = np.vstack( (X_real, X_syn) ), np.vstack( (y_real, y_syn) )

            d_loss, d_accuracy  = d_model.train_on_batch(X, y)



            X_gan = generate_latent_points(latent_dim, n_batch)
            y_gan = np.ones((n_batch, 1))

            g_loss = gan_model.train_on_batch(X_gan, y_gan)

            #print('>%d, %d/%d, d=%.3f, g=' % (i+1, j+1, batch_per_epoch, d_loss_real + d_loss_fake), end='')
            print(', '.join(['%.3f' % loss for loss in g_loss]))


In [ ]:
latent_dim = 100

d_model = Discriminator()
g_model = Generator(latent_dim)
gan_model = GAN(g_model, d_model)
dataset = load_real_samples()


In [ ]:
dataset.shape
dataset.shape[0]/256

In [ ]:
train_gen_dis(g_model, d_model, gan_model, dataset, latent_dim, n_epoches = 40, n_batch = 256)

In [ ]:
latent_dim = 100
n_batch = 128
half_batch = 128


X_gan = generate_latent_points(latent_dim, n_batch)
X_syn, y_syn = generate_synthetic_samples(g_model, latent_dim, half_batch)

In [ ]:
fig, axes = plt.subplots(3, 5, figsize=(10, 6))
axes = axes.flatten()
for i in range(15):
    axes[i].axis('off')
    axes[i].imshow(X_syn[i, :, :, 0])


plt.tight_layout()
plt.show()